In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Image Size Check

In [ ]:
from PIL import Image
import os

def calculate_image_size_statistics(dataset_folders):
    min_width = float('inf')
    min_height = float('inf')
    max_width = 0
    max_height = 0
    total_width = 0
    total_height = 0
    total_images = 0

    for folder in dataset_folders:
        for filename in os.listdir(folder):
            image_path = os.path.join(folder, filename)

            try:
                # 이미지 열기
                img = Image.open(image_path)

                # 이미지 크기 측정
                width, height = img.size
                total_width += width
                total_height += height
                total_images += 1

                # 최소 및 최대 크기 업데이트
                min_width = min(min_width, width)
                min_height = min(min_height, height)
                max_width = max(max_width, width)
                max_height = max(max_height, height)

            except Exception as e:
                print(f"Error processing {image_path}: {e}")

    # 이미지 평균 크기 계산
    average_width = total_width / total_images
    average_height = total_height / total_images

    return {
        'average_width': average_width,
        'average_height': average_height,
        'min_width': min_width,
        'min_height': min_height,
        'max_width': max_width,
        'max_height': max_height
    }

# 데이터셋 폴더 경로
dataset_folders = [
    "/content/drive/MyDrive/Colab Notebooks/finalproject_dataset/disease",
    "/content/drive/MyDrive/Colab Notebooks/finalproject_dataset/healthy"
]

# 이미지 크기 통계 계산
image_size_statistics = calculate_image_size_statistics(dataset_folders)

# 결과 출력
print(f"전체 데이터셋의 평균 이미지 폭: {image_size_statistics['average_width']}")
print(f"전체 데이터셋의 평균 이미지 높이: {image_size_statistics['average_height']}")
print(f"전체 데이터셋의 최소 이미지 폭: {image_size_statistics['min_width']}")
print(f"전체 데이터셋의 최소 이미지 높이: {image_size_statistics['min_height']}")
print(f"전체 데이터셋의 최대 이미지 폭: {image_size_statistics['max_width']}")
print(f"전체 데이터셋의 최대 이미지 높이: {image_size_statistics['max_height']}")


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (127401984 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


전체 데이터셋의 평균 이미지 폭: 1391.0697674418604
전체 데이터셋의 평균 이미지 높이: 1531.1834625322997
전체 데이터셋의 최소 이미지 폭: 152
전체 데이터셋의 최소 이미지 높이: 152
전체 데이터셋의 최대 이미지 폭: 13824
전체 데이터셋의 최대 이미지 높이: 10260


# Main

In [ ]:
import os
import cv2
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
import numpy as np

from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import AdamW
from keras.regularizers import l2

from keras.callbacks import EarlyStopping


In [ ]:
# 증강x, baseline
def load_images_and_labels(data_dir):
    images = []
    labels = []

    for folder_name in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, folder_name)
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                img_path = os.path.join(folder_path, filename)
                img = cv2.imread(img_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # 이미지를 RGB 형식으로 변환
                img = cv2.resize(img, (128, 128), interpolation=cv2.INTER_LINEAR)  # 이미지 크기 조절 / 보간법 사용

                # 이미지 데이터를 리스트에 추가
                images.append(img)
                # 레이블을 리스트에 추가
                labels.append(folder_name)

    return shuffle(images, labels, random_state=42)


In [ ]:
# 랜덤 증강o

def load_images_and_labels(data_dir):
    images = []
    labels = []

    # 랜덤으로 선택할 증강 리스트
    augmentation_options = [
        {'name': 'rotation', 'param': 40},
        {'name': 'width_shift', 'param': 0.2},
        {'name': 'height_shift', 'param': 0.2},
        {'name': 'shear', 'param': 0.2},
        {'name': 'zoom', 'param': 0.2},
        {'name': 'horizontal_flip', 'param': True},
    ]

    # 데이터 증강을 위한 ImageDataGenerator 정의
    datagen = ImageDataGenerator()

    for folder_name in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, folder_name)
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                img_path = os.path.join(folder_path, filename)
                img = cv2.imread(img_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (128, 128), interpolation=cv2.INTER_LINEAR)

                # 원본 이미지를 리스트에 추가
                images.append(img)
                labels.append(folder_name)

                # 랜덤으로 선택한 증강 옵션을 적용
                selected_augmentations = random.sample(augmentation_options, k=random.randint(0, len(augmentation_options)))
                for aug_option in selected_augmentations:
                    if aug_option['name'] == 'rotation':
                        img = ImageDataGenerator(rotation_range=aug_option['param']).random_transform(img)
                    elif aug_option['name'] == 'width_shift':
                        img = ImageDataGenerator(width_shift_range=aug_option['param']).random_transform(img)
                    elif aug_option['name'] == 'height_shift':
                        img = ImageDataGenerator(height_shift_range=aug_option['param']).random_transform(img)
                    elif aug_option['name'] == 'shear':
                        img = ImageDataGenerator(shear_range=aug_option['param']).random_transform(img)
                    elif aug_option['name'] == 'zoom':
                        img = ImageDataGenerator(zoom_range=aug_option['param']).random_transform(img)
                    elif aug_option['name'] == 'horizontal_flip':
                        img = ImageDataGenerator(horizontal_flip=aug_option['param']).random_transform(img)

                    # 증강된 이미지를 리스트에 추가
                    images.append(img)
                    labels.append(folder_name)
    return shuffle(images, labels, random_state=42)

# Class Weight



In [ ]:
# 데이터 불러오기
data_dir = "/content/drive/MyDrive/Colab Notebooks/finalproject_dataset"
X, y = load_images_and_labels(data_dir)

# 레이블 인코딩
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y, num_classes=2)  # 클래스 개수에 따라 수정

# 이미지 데이터를 전처리
X = np.array(X)
y = np.array(y)

# K-fold 교차 검증을 위한 K 값 설정
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)

# 모델 생성 함수
def create_vgg16_model():
    model = Sequential()

    # Block 1
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 2
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 3
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 4
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 5
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Dropout(0.5))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Dropout(0.5))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    # Classification block
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=AdamW(learning_rate=0.0001), metrics=['accuracy'])

    return model


# 전체 k-fold에 대한 accuracy를 저장할 리스트
all_accuracies = []

# K-fold 교차 검증 수행
for train_index, test_index in k_fold.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = create_vgg16_model()

    class_weight = {0: 0.15, 1: 0.85}

    # 모델 훈련
    model.fit(X_train, y_train, epochs=50, batch_size=128, validation_data=(X_test, y_test), class_weight=class_weight)

    # 모델 평가
    eval_result = model.evaluate(X_test, y_test)
    print(f"Test Loss: {eval_result[0]}, Test Accuracy: {eval_result[1]}")

    # 평가 결과를 리스트에 추가
    all_accuracies.append(eval_result[1])

# 전체 k-fold에 대한 평균 accuracy 계산
average_accuracy = np.mean(all_accuracies)
print(f"\nAverage Accuracy Across All Folds: {average_accuracy}")

Epoch 1/50
10/10 [==============================] - 22s 2s/step - loss: 0.2815 - accuracy: 0.6713 - val_loss: 0.6215 - val_accuracy: 0.6591
Epoch 2/50
10/10 [==============================] - 7s 688ms/step - loss: 0.1523 - accuracy: 0.6916 - val_loss: 0.6016 - val_accuracy: 0.6591
Epoch 3/50
10/10 [==============================] - 7s 689ms/step - loss: 0.1447 - accuracy: 0.6916 - val_loss: 0.5894 - val_accuracy: 0.6591
Epoch 4/50
10/10 [==============================] - 7s 681ms/step - loss: 0.1235 - accuracy: 0.6916 - val_loss: 0.5408 - val_accuracy: 0.6591
Epoch 5/50
10/10 [==============================] - 7s 670ms/step - loss: 0.1218 - accuracy: 0.6916 - val_loss: 0.5452 - val_accuracy: 0.6591
Epoch 6/50
10/10 [==============================] - 7s 663ms/step - loss: 0.1088 - accuracy: 0.6916 - val_loss: 0.5016 - val_accuracy: 0.6591
Epoch 7/50
10/10 [==============================] - 7s 658ms/step - loss: 0.1095 - accuracy: 0.6916 - val_loss: 0.5483 - val_accuracy: 0.6591
Epoch 8/

# UpSampling by SMOTE



undersampling 하지 않은 이유 -> 안 그래도 적은 데이터에 대해 데이터 수를 더 줄이는 것은 옳지 않다고 판단

overfitting 가능성 증가 -> epoch 줄임

In [ ]:
import imblearn
import matplotlib.pyplot as plt
from numpy import where
from imblearn.over_sampling import SMOTE
from collections import Counter

Epoch 50 -> 30

In [ ]:
# 데이터 불러오기
data_dir = "/content/drive/MyDrive/Colab Notebooks/finalproject_dataset"
X, y = load_images_and_labels(data_dir)

# 레이블 인코딩
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y, num_classes=2)  # 클래스 개수에 따라 수정

# 이미지 데이터를 전처리
# Flatten each image if needed
X = np.array(X)
y = np.array(y)

X_flatten = X.reshape(X.shape[0], -1)

# Apply SMOTE
smt = SMOTE()
X_new, y_new = smt.fit_resample(X_flatten, y)

X_new_reshaped = X_new.reshape(X_new.shape[0], 128, 128, 3)

# K-fold 교차 검증을 위한 K 값 설정
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)

# 모델 생성 함수
def create_vgg16_model():
    model = Sequential()

    # Block 1
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 2
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 3
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 4
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 5
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Dropout(0.5))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Dropout(0.5))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    # Classification block
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=AdamW(learning_rate=0.0001), metrics=['accuracy'])

    return model


# 전체 k-fold에 대한 accuracy를 저장할 리스트
all_accuracies = []

# K-fold 교차 검증 수행
for train_index, test_index in k_fold.split(X_new_reshaped):
    X_train, X_test = X_new_reshaped[train_index], X_new_reshaped[test_index]
    y_train, y_test = y_new[train_index], y_new[test_index]

    model = create_vgg16_model()


    # 모델 훈련
    model.fit(X_train, y_train, epochs=30, batch_size=128, validation_data=(X_test, y_test)) # 오버피팅 가능성이 있기 때문에 epoch 줄임

    # 모델 평가
    eval_result = model.evaluate(X_test, y_test)
    print(f"Test Loss: {eval_result[0]}, Test Accuracy: {eval_result[1]}")

    # 평가 결과를 리스트에 추가
    all_accuracies.append(eval_result[1])

# 전체 k-fold에 대한 평균 accuracy 계산
average_accuracy = np.mean(all_accuracies)
print(f"\nAverage Accuracy Across All Folds: {average_accuracy}")

Epoch 1/30
14/14 [==============================] - 47s 2s/step - loss: 0.8358 - accuracy: 0.4920 - val_loss: 0.6906 - val_accuracy: 0.5364
Epoch 2/30
14/14 [==============================] - 9s 628ms/step - loss: 0.6949 - accuracy: 0.5330 - val_loss: 0.6869 - val_accuracy: 0.5977
Epoch 3/30
14/14 [==============================] - 9s 631ms/step - loss: 0.6217 - accuracy: 0.5563 - val_loss: 0.5530 - val_accuracy: 0.7614
Epoch 4/30
14/14 [==============================] - 9s 634ms/step - loss: 0.5188 - accuracy: 0.7309 - val_loss: 0.5181 - val_accuracy: 0.7818
Epoch 5/30
14/14 [==============================] - 9s 637ms/step - loss: 0.4408 - accuracy: 0.8180 - val_loss: 0.5015 - val_accuracy: 0.7159
Epoch 6/30
14/14 [==============================] - 9s 637ms/step - loss: 0.3953 - accuracy: 0.8311 - val_loss: 0.3594 - val_accuracy: 0.8455
Epoch 7/30
14/14 [==============================] - 9s 639ms/step - loss: 0.3192 - accuracy: 0.8578 - val_loss: 0.3192 - val_accuracy: 0.8659
Epoch 8/

BatchNormalization O / Epoch 30

In [ ]:
# 데이터 불러오기
data_dir = "/content/drive/MyDrive/Colab Notebooks/finalproject_dataset"
X, y = load_images_and_labels(data_dir)

# 레이블 인코딩
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y, num_classes=2)  # 클래스 개수에 따라 수정

# 이미지 데이터를 전처리
# Flatten each image if needed
X = np.array(X)
y = np.array(y)

X_flatten = X.reshape(X.shape[0], -1)

# Apply SMOTE
smt = SMOTE()
X_new, y_new = smt.fit_resample(X_flatten, y)

X_new_reshaped = X_new.reshape(X_new.shape[0], 128, 128, 3)

# K-fold 교차 검증을 위한 K 값 설정
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)


# 모델 생성 함수
def create_vgg16_model_with_upsampling():
    model = Sequential()

    # Block 1
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 2
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 3
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 4
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 5
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))  # Adding Dropout with a dropout rate of 0.5

    # Classification block
    model.add(Flatten())
    model.add(Dense(4096, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))  # Adding Dropout with a dropout rate of 0.5
    model.add(Dense(4096, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))  # Adding Dropout with a dropout rate of 0.5
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=AdamW(learning_rate=0.0001), metrics=['accuracy'])

    return model

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 전체 k-fold에 대한 accuracy를 저장할 리스트
all_accuracies = []

# K-fold 교차 검증 수행
for train_index, test_index in k_fold.split(X_new_reshaped):
    X_train, X_test = X_new_reshaped[train_index], X_new_reshaped[test_index]
    y_train, y_test = y_new[train_index], y_new[test_index]

    model = create_vgg16_model_with_upsampling()


    # 모델 훈련
    model.fit(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_test, y_test)) # 오버피팅 가능성이 있기 때문에 epoch 줄임

    # 모델 평가
    eval_result = model.evaluate(X_test, y_test)
    print(f"Test Loss: {eval_result[0]}, Test Accuracy: {eval_result[1]}")

    # 평가 결과를 리스트에 추가
    all_accuracies.append(eval_result[1])

# 전체 k-fold에 대한 평균 accuracy 계산
average_accuracy = np.mean(all_accuracies)
print(f"\nAverage Accuracy Across All Folds: {average_accuracy}")

Epoch 1/30
27/27 [==============================] - 35s 740ms/step - loss: 92.9131 - accuracy: 0.8148 - val_loss: 89.7112 - val_accuracy: 0.5280
Epoch 2/30
27/27 [==============================] - 12s 437ms/step - loss: 85.4968 - accuracy: 0.9100 - val_loss: 83.2460 - val_accuracy: 0.4977
Epoch 3/30
27/27 [==============================] - 12s 438ms/step - loss: 78.1490 - accuracy: 0.9369 - val_loss: 76.5865 - val_accuracy: 0.5000
Epoch 4/30
27/27 [==============================] - 12s 440ms/step - loss: 71.0741 - accuracy: 0.9702 - val_loss: 70.4570 - val_accuracy: 0.4930
Epoch 5/30
27/27 [==============================] - 12s 441ms/step - loss: 64.4054 - accuracy: 0.9714 - val_loss: 64.2630 - val_accuracy: 0.4977
Epoch 6/30
27/27 [==============================] - 12s 442ms/step - loss: 58.1468 - accuracy: 0.9842 - val_loss: 56.9918 - val_accuracy: 0.5210
Epoch 7/30
27/27 [==============================] - 12s 443ms/step - loss: 52.3892 - accuracy: 0.9772 - val_loss: 52.3755 - val_ac

Epoch 40

In [ ]:
# 데이터 불러오기
data_dir = "/content/drive/MyDrive/Colab Notebooks/finalproject_dataset"
X, y = load_images_and_labels(data_dir)

# 레이블 인코딩
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y, num_classes=2)  # 클래스 개수에 따라 수정

# 이미지 데이터를 전처리
# Flatten each image if needed
X = np.array(X)
y = np.array(y)

X_flatten = X.reshape(X.shape[0], -1)

# Apply SMOTE
smt = SMOTE()
X_new, y_new = smt.fit_resample(X_flatten, y)

X_new_reshaped = X_new.reshape(X_new.shape[0], 128, 128, 3)

# K-fold 교차 검증을 위한 K 값 설정
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)

# 모델 생성 함수
def create_vgg16_model():
    model = Sequential()

    # Block 1
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 2
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 3
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 4
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 5
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Dropout(0.5))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Dropout(0.5))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    # Classification block
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=AdamW(learning_rate=0.0001), metrics=['accuracy'])

    return model


# 전체 k-fold에 대한 accuracy를 저장할 리스트
all_accuracies = []

# K-fold 교차 검증 수행
for train_index, test_index in k_fold.split(X_new_reshaped):
    X_train, X_test = X_new_reshaped[train_index], X_new_reshaped[test_index]
    y_train, y_test = y_new[train_index], y_new[test_index]

    model = create_vgg16_model()


    # 모델 훈련
    model.fit(X_train, y_train, epochs=40, batch_size=128, validation_data=(X_test, y_test)) # 오버피팅 가능성이 있기 때문에 epoch 줄임

    # 모델 평가
    eval_result = model.evaluate(X_test, y_test)
    print(f"Test Loss: {eval_result[0]}, Test Accuracy: {eval_result[1]}")

    # 평가 결과를 리스트에 추가
    all_accuracies.append(eval_result[1])

# 전체 k-fold에 대한 평균 accuracy 계산
average_accuracy = np.mean(all_accuracies)
print(f"\nAverage Accuracy Across All Folds: {average_accuracy}")

Epoch 1/40
14/14 [==============================] - 43s 1s/step - loss: 1.1552 - accuracy: 0.4971 - val_loss: 0.6928 - val_accuracy: 0.4838
Epoch 2/40
14/14 [==============================] - 9s 624ms/step - loss: 0.6779 - accuracy: 0.5041 - val_loss: 0.6811 - val_accuracy: 0.6157
Epoch 3/40
14/14 [==============================] - 9s 627ms/step - loss: 0.6167 - accuracy: 0.5777 - val_loss: 0.6168 - val_accuracy: 0.6806
Epoch 4/40
14/14 [==============================] - 9s 630ms/step - loss: 0.5644 - accuracy: 0.7709 - val_loss: 0.6512 - val_accuracy: 0.7222
Epoch 5/40
14/14 [==============================] - 9s 635ms/step - loss: 0.5374 - accuracy: 0.7947 - val_loss: 0.5584 - val_accuracy: 0.8079
Epoch 6/40
14/14 [==============================] - 9s 641ms/step - loss: 0.4901 - accuracy: 0.8347 - val_loss: 0.5234 - val_accuracy: 0.8241
Epoch 7/40
14/14 [==============================] - 9s 644ms/step - loss: 0.4477 - accuracy: 0.8590 - val_loss: 0.4713 - val_accuracy: 0.8241
Epoch 8/

# Data Augmentation for Minor class

class 1이 minor class

Random으로 여러 개의 augmentation을 minor class에 적용

In [ ]:
import os
import cv2
import random
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_images_and_labels(data_dir, minority_class):
    images = []
    labels = []

    # 랜덤으로 선택할 증강 리스트
    augmentation_options = [
        {'name': 'rotation', 'param': 40},
        {'name': 'width_shift', 'param': 0.2},
        {'name': 'height_shift', 'param': 0.2},
        {'name': 'shear', 'param': 0.2},
        {'name': 'zoom', 'param': 0.2},
        {'name': 'horizontal_flip', 'param': True},
    ]

    # 데이터 증강을 위한 ImageDataGenerator 정의
    datagen = ImageDataGenerator()

    for folder_name in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, folder_name)
        if os.path.isdir(folder_path):
            is_minority_class = (folder_name == minority_class)

            for filename in os.listdir(folder_path):
                img_path = os.path.join(folder_path, filename)
                img = cv2.imread(img_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (128, 128), interpolation=cv2.INTER_LINEAR)

                # 원본 이미지를 리스트에 추가
                images.append(img)
                labels.append(folder_name)

                # 랜덤으로 선택한 증강 옵션을 적용 (for the minority class)
                if is_minority_class:
                    selected_augmentations = random.sample(augmentation_options, k=random.randint(0, len(augmentation_options)))
                    for aug_option in selected_augmentations:
                        if aug_option['name'] == 'rotation':
                            img = ImageDataGenerator(rotation_range=aug_option['param']).random_transform(img)
                        elif aug_option['name'] == 'width_shift':
                            img = ImageDataGenerator(width_shift_range=aug_option['param']).random_transform(img)
                        elif aug_option['name'] == 'height_shift':
                            img = ImageDataGenerator(height_shift_range=aug_option['param']).random_transform(img)
                        elif aug_option['name'] == 'shear':
                            img = ImageDataGenerator(shear_range=aug_option['param']).random_transform(img)
                        elif aug_option['name'] == 'zoom':
                            img = ImageDataGenerator(zoom_range=aug_option['param']).random_transform(img)
                        elif aug_option['name'] == 'horizontal_flip':
                            img = ImageDataGenerator(horizontal_flip=aug_option['param']).random_transform(img)

                        # 증강된 이미지를 리스트에 추가
                        images.append(img)
                        labels.append(folder_name)
                else:
                    # For majority class, you can optionally choose not to apply augmentation
                    pass

    return shuffle(images, labels, random_state=42)



In [ ]:
# 데이터 불러오기
data_dir = "/content/drive/MyDrive/Colab Notebooks/finalproject_dataset"
X, y = load_images_and_labels(data_dir, 1)

# 레이블 인코딩
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y, num_classes=2)  # 클래스 개수에 따라 수정

# 이미지 데이터를 전처리
X = np.array(X)
y = np.array(y)

# K-fold 교차 검증을 위한 K 값 설정
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)

# 모델 생성 함수
def create_vgg16_model():
    model = Sequential()

    # Block 1
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 2
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 3
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 4
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 5
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Dropout(0.5))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Dropout(0.5))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    # Classification block
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=AdamW(learning_rate=0.0001), metrics=['accuracy'])

    return model


# 전체 k-fold에 대한 accuracy를 저장할 리스트
all_accuracies = []

# K-fold 교차 검증 수행
for train_index, test_index in k_fold.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = create_vgg16_model()


    # 모델 훈련
    model.fit(X_train, y_train, epochs=50, batch_size=128, validation_data=(X_test, y_test))

    # 모델 평가
    eval_result = model.evaluate(X_test, y_test)
    print(f"Test Loss: {eval_result[0]}, Test Accuracy: {eval_result[1]}")

    # 평가 결과를 리스트에 추가
    all_accuracies.append(eval_result[1])

# 전체 k-fold에 대한 평균 accuracy 계산
average_accuracy = np.mean(all_accuracies)
print(f"\nAverage Accuracy Across All Folds: {average_accuracy}")

Epoch 1/50
3/3 [==============================] - 15s 5s/step - loss: 1.4472 - accuracy: 0.6408 - val_loss: 0.6892 - val_accuracy: 0.6667
Epoch 2/50
3/3 [==============================] - 2s 520ms/step - loss: 0.6839 - accuracy: 0.5502 - val_loss: 0.6846 - val_accuracy: 0.6667
Epoch 3/50
3/3 [==============================] - 2s 523ms/step - loss: 0.6623 - accuracy: 0.6278 - val_loss: 0.6494 - val_accuracy: 0.6667
Epoch 4/50
3/3 [==============================] - 2s 523ms/step - loss: 0.6404 - accuracy: 0.6893 - val_loss: 0.6569 - val_accuracy: 0.6667
Epoch 5/50
3/3 [==============================] - 2s 524ms/step - loss: 0.6067 - accuracy: 0.6893 - val_loss: 0.6658 - val_accuracy: 0.6667
Epoch 6/50
3/3 [==============================] - 2s 523ms/step - loss: 0.6127 - accuracy: 0.6893 - val_loss: 0.6593 - val_accuracy: 0.6667
Epoch 7/50
3/3 [==============================] - 2s 526ms/step - loss: 0.5911 - accuracy: 0.6893 - val_loss: 0.6670 - val_accuracy: 0.6667
Epoch 8/50
3/3 [======

Random augmentation 1개 적용

데이터 양이 2배 가까이 차이나기에, 이를 균형 맞추기 위해 augmentation 1개만 적용해서 데이터를 2배로 만듦

In [ ]:
import os
import cv2
import random
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_images_and_labels(data_dir, minority_class):
    images = []
    labels = []

    # 랜덤으로 선택할 증강 리스트
    augmentation_options = [
        {'name': 'rotation', 'param': 40},
        {'name': 'width_shift', 'param': 0.2},
        {'name': 'height_shift', 'param': 0.2},
        {'name': 'shear', 'param': 0.2},
        {'name': 'zoom', 'param': 0.2},
        {'name': 'horizontal_flip', 'param': True},
    ]

    # 데이터 증강을 위한 ImageDataGenerator 정의
    datagen = ImageDataGenerator()

    for folder_name in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, folder_name)
        if os.path.isdir(folder_path):
            is_minority_class = (folder_name == minority_class)

            for filename in os.listdir(folder_path):
                img_path = os.path.join(folder_path, filename)
                img = cv2.imread(img_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (128, 128), interpolation=cv2.INTER_LINEAR)

                # 원본 이미지를 리스트에 추가
                images.append(img)
                labels.append(folder_name)

                # 랜덤으로 선택한 증강 옵션을 적용 (for the minority class)
                if is_minority_class:
                    random_augmentation = random.choice(augmentation_options)
                    aug_name, aug_param = random_augmentation['name'], random_augmentation['param']
                    img = apply_augmentation(img, aug_name, aug_param)

                    # 증강된 이미지를 리스트에 추가
                    images.append(img)
                    labels.append(folder_name)

    return shuffle(images, labels, random_state=42)

def apply_augmentation(img, aug_name, aug_param):
    datagen = ImageDataGenerator()
    if aug_name == 'rotation':
        img = datagen.apply_transform(img, {'theta': aug_param})
    elif aug_name == 'width_shift':
        img = datagen.apply_transform(img, {'tx': aug_param * img.shape[1]})
    elif aug_name == 'height_shift':
        img = datagen.apply_transform(img, {'ty': aug_param * img.shape[0]})
    elif aug_name == 'shear':
        img = datagen.apply_transform(img, {'shear': aug_param})
    elif aug_name == 'zoom':
        img = datagen.apply_transform(img, {'zx': 1 + aug_param, 'zy': 1 + aug_param})
    elif aug_name == 'horizontal_flip':
        img = datagen.apply_transform(img, {'flip_horizontal': aug_param})

    return img

In [ ]:
# 데이터 불러오기
data_dir = "/content/drive/MyDrive/Colab Notebooks/finalproject_dataset"
X, y = load_images_and_labels(data_dir, 1)

# 레이블 인코딩
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y, num_classes=2)  # 클래스 개수에 따라 수정

# 이미지 데이터를 전처리
X = np.array(X)
y = np.array(y)

# K-fold 교차 검증을 위한 K 값 설정
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)

# 모델 생성 함수
def create_vgg16_model():
    model = Sequential()

    # Block 1
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 2
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 3
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 4
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 5
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Dropout(0.5))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Dropout(0.5))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    # Classification block
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=AdamW(learning_rate=0.0001), metrics=['accuracy'])

    return model


# 전체 k-fold에 대한 accuracy를 저장할 리스트
all_accuracies = []

# K-fold 교차 검증 수행
for train_index, test_index in k_fold.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = create_vgg16_model()


    # 모델 훈련
    model.fit(X_train, y_train, epochs=50, batch_size=128, validation_data=(X_test, y_test))

    # 모델 평가
    eval_result = model.evaluate(X_test, y_test)
    print(f"Test Loss: {eval_result[0]}, Test Accuracy: {eval_result[1]}")

    # 평가 결과를 리스트에 추가
    all_accuracies.append(eval_result[1])

# 전체 k-fold에 대한 평균 accuracy 계산
average_accuracy = np.mean(all_accuracies)
print(f"\nAverage Accuracy Across All Folds: {average_accuracy}")

Epoch 1/50
3/3 [==============================] - 31s 5s/step - loss: 1.6414 - accuracy: 0.6149 - val_loss: 0.6518 - val_accuracy: 0.6667
Epoch 2/50
3/3 [==============================] - 2s 496ms/step - loss: 0.6308 - accuracy: 0.6893 - val_loss: 0.6754 - val_accuracy: 0.6667
Epoch 3/50
3/3 [==============================] - 2s 495ms/step - loss: 0.6157 - accuracy: 0.6893 - val_loss: 0.6702 - val_accuracy: 0.6667
Epoch 4/50
3/3 [==============================] - 2s 497ms/step - loss: 0.6046 - accuracy: 0.6893 - val_loss: 0.6589 - val_accuracy: 0.6667
Epoch 5/50
3/3 [==============================] - 2s 499ms/step - loss: 0.5964 - accuracy: 0.6893 - val_loss: 0.6644 - val_accuracy: 0.6667
Epoch 6/50
3/3 [==============================] - 2s 500ms/step - loss: 0.6072 - accuracy: 0.6893 - val_loss: 0.6714 - val_accuracy: 0.6667
Epoch 7/50
3/3 [==============================] - 2s 499ms/step - loss: 0.5961 - accuracy: 0.6893 - val_loss: 0.6541 - val_accuracy: 0.6667
Epoch 8/50
3/3 [======